# Imports

In [2]:
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
# 3D plotting
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker

import numpy as np
import time
from pyDOE import lhs         # Latin Hypercube Sampling
import scipy.io               # Loading .mat matlab data 

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print("Running this on", device)

if device == 'cuda': 
    print(torch.cuda.get_device_name()) 

Running this on cpu


# Main loop

In [3]:
def main_loop(N_u, N_f, num_layers, num_neurons):
     
    nu = 0.01/np.pi

    layers = np.concatenate([[2], num_neurons*np.ones(num_layers), [1]]).astype(int).tolist()
    
    data = scipy.io.loadmat('Data/burgers_shock.mat')
    
    t = data['t'].flatten()[:,None]
    x = data['x'].flatten()[:,None]
    Exact = np.real(data['usol']).T
    
    X, T = np.meshgrid(x,t)
    
    X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
    u_star = Exact.flatten()[:,None]              

    # Doman bounds
    lb = X_star.min(0)
    ub = X_star.max(0)    
        
    xx1 = np.hstack((X[0:1,:].T, T[0:1,:].T))
    uu1 = Exact[0:1,:].T
    xx2 = np.hstack((X[:,0:1], T[:,0:1]))
    uu2 = Exact[:,0:1]
    xx3 = np.hstack((X[:,-1:], T[:,-1:]))
    uu3 = Exact[:,-1:]
    
    X_u_train = np.vstack([xx1, xx2, xx3])
    X_f_train = lb + (ub-lb)*lhs(2, N_f)
    X_f_train = np.vstack((X_f_train, X_u_train))
    u_train = np.vstack([uu1, uu2, uu3])
    
    idx = np.random.choice(X_u_train.shape[0], N_u, replace=False)
    X_u_train = X_u_train[idx, :]
    u_train = u_train[idx,:]
        
    # model = PhysicsInformedNN(X_u_train, u_train, X_f_train, layers, lb, ub, nu)
    
    # start_time = time.time()                
    # model.train()
    # elapsed = time.time() - start_time                
    # print('Training time: %.4f' % (elapsed))
    
    # u_pred, f_pred = model.predict(X_star)
            
    # error_u = np.linalg.norm(u_star-u_pred,2)/np.linalg.norm(u_star,2)
     
    error_u = 0   
    
    return error_u

# Test

In [5]:
main_loop(20, 2000, 8, 40)

0

# Main

In [ ]:
if __name__ == "__main__": 
        
    # Training data
    N_u = [20, 40, 60, 80, 100, 200]
    
    # Collocation points
    N_f = [2000, 4000, 6000, 7000, 8000, 10000]
    
    num_layers = [2,4,6,8]
    num_neurons = [10,20,40]    
    
    error_table_1 = np.zeros((len(N_u), len(N_f)))
    error_table_2 = np.zeros((len(num_layers), len(num_neurons)))
 
    # Used to calculate errors across varying number of training-collocation and varying numbur of layers and neurons

    for i in range(len(N_u)):
        for j in range(len(N_f)):
            error_table_1[i,j] = main_loop(N_u[i], N_f[j], num_layers[-1], num_neurons[-1])
            
    for i in range(len(num_layers)):
        for j in range(len(num_neurons)):
            error_table_2[i,j] = main_loop(N_u[-1], N_f[-1], num_layers[i], num_neurons[j])
            
    np.savetxt('Tables/error_table_1.csv', error_table_1, delimiter=' & ', fmt='$%.2e$', newline=' \\\\\n')
    np.savetxt('Tables/error_table_2.csv', error_table_2, delimiter=' & ', fmt='$%.2e$', newline=' \\\\\n')
